# XGBoost Multi-Output Regression with Nested CV
This notebook trains a global XGBoost model to predict **HI, TMAX, RH** using multi-output regression with **nested cross-validation**, chronological splits, and early stopping.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, TimeSeriesSplit, RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb

# Ensure output folders
os.makedirs("models", exist_ok=True)


In [ ]:
# Load all station CSVs from merged_datasets folder
folder = "merged_datasets"
all_data = []

for file in os.listdir(folder):
    if file.endswith(".csv"):
        station = file.replace(".csv", "")
        df = pd.read_csv(os.path.join(folder, file))
        df["Station"] = station
        all_data.append(df)

data = pd.concat(all_data, ignore_index=True)

print("Shape:", data.shape)
data.head()


In [ ]:
# Define features and targets
target_cols = ["HI", "TMAX", "RH"]
feature_cols = [c for c in data.columns if c not in target_cols + ["Station", "YEAR", "MONTH", "DAY"]]

X = data[feature_cols]
y = data[target_cols]
stations = data["Station"]


In [ ]:
# Chronological 80-10-10 split
n = len(data)
train_end = int(n * 0.8)
val_end = int(n * 0.9)

X_train, y_train = X.iloc[:train_end], y.iloc[:train_end]
X_val, y_val = X.iloc[train_end:val_end], y.iloc[train_end:val_end]
X_test, y_test = X.iloc[val_end:], y.iloc[val_end:]
stations_val = stations.iloc[train_end:val_end]
stations_test = stations.iloc[val_end:]

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


In [ ]:
# Define base estimator
xgb_est = xgb.XGBRegressor(
    objective="reg:squarederror",
    eval_metric="rmse",
    tree_method="hist",
    random_state=42
)

multi_est = MultiOutputRegressor(xgb_est)

# Hyperparameter grid for RandomizedSearchCV
param_grid = {
    "estimator__n_estimators": [200, 500, 1000],
    "estimator__learning_rate": [0.01, 0.05, 0.1],
    "estimator__max_depth": [3, 5, 7],
    "estimator__subsample": [0.7, 0.8, 1.0],
    "estimator__colsample_bytree": [0.7, 0.8, 1.0]
}

# Inner CV
tscv = TimeSeriesSplit(n_splits=3)

search = RandomizedSearchCV(
    estimator=multi_est,
    param_distributions=param_grid,
    n_iter=10,
    scoring="neg_mean_squared_error",
    cv=tscv,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

search.fit(X_train, y_train)

print("Best params:", search.best_params_)


In [ ]:
best_params = search.best_params_
best_est = xgb.XGBRegressor(
    objective="reg:squarederror",
    eval_metric="rmse",
    tree_method="hist",
    random_state=42,
    **{k.replace("estimator__", ""): v for k,v in best_params.items()}
)

final_model = MultiOutputRegressor(best_est)

# Use validation set for early stopping
final_model.fit(
    X_train, y_train,
    **{"estimator__eval_set":[(X_val, y_val)], "estimator__early_stopping_rounds":20, "estimator__verbose":False}
)

# Save model
final_model.estimators_[0].save_model("models/xgb_model.json")


In [ ]:
def evaluate_predictions(y_true, y_pred, stations_subset):
    results = []
    for st in stations_subset.unique():
        idx = stations_subset == st
        rmse = np.sqrt(mean_squared_error(y_true[idx], y_pred[idx]))
        r2 = r2_score(y_true[idx], y_pred[idx])
        results.append({"Station": st, "RMSE": rmse, "R2": r2})
    return pd.DataFrame(results)

# Validation results
val_pred = final_model.predict(X_val)
df_val_metrics = evaluate_predictions(y_val, val_pred, stations_val)

# Test results
test_pred = final_model.predict(X_test)
df_test_metrics = evaluate_predictions(y_test, test_pred, stations_test)

print("Validation Metrics:")
display(df_val_metrics)

print("Test Metrics:")
display(df_test_metrics)


In [ ]:
# Plot actual vs predicted per station (test set)
for st in stations_test.unique():
    idx = stations_test == st
    plt.figure(figsize=(10,4))
    plt.plot(y_test[idx].values[:,0], label="Actual HI")
    plt.plot(test_pred[idx][:,0], label="Pred HI")
    plt.title(f"Station {st} - HI Prediction (Test)")
    plt.legend()
    plt.show()


In [ ]:

# --- AUTO-ADDED: Forecast HI for t+1 and t+2, display metrics as tables ---
import warnings
warnings.filterwarnings("ignore")

import os, sys
import numpy as np, pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
try:
    from xgboost import XGBRegressor
    has_xgb = True
except Exception as e:
    XGBRegressor = None
    has_xgb = False

# 1) Get df
try:
    df
    print("Using existing DataFrame `df` from the notebook kernel.")
except NameError:
    raise RuntimeError("No DataFrame `df` present. Please run the notebook's data-loading cells first.")

# 2) Identify station and datetime columns
cols = df.columns.tolist()

def find_column(candidates):
    for cand in candidates:
        for c in cols:
            if cand.lower() == c.lower():
                return c
    for cand in candidates:
        for c in cols:
            if cand.lower() in c.lower():
                return c
    return None

station_col = find_column(['station','station_id','stn','site','stationid','station_code'])
datetime_col = find_column(['datetime','date','time','timestamp','obs_time','datetime_utc'])

if station_col is None:
    raise RuntimeError("Could not detect a station column. Columns found: " + ", ".join(cols))

if datetime_col is not None:
    try:
        df[datetime_col] = pd.to_datetime(df[datetime_col])
    except Exception:
        pass

# 3) Find HI column
hi_col = find_column(['HI','heat_index','heat index','heatindex','hi'])
if hi_col is None:
    raise RuntimeError("Could not detect a Heat Index (HI) column. Columns found: " + ", ".join(cols))

print(f"Detected station column: {station_col}, datetime: {datetime_col}, HI column: {hi_col}")

# sort by station and datetime if available
if datetime_col is not None:
    df = df.sort_values(by=[station_col, datetime_col]).reset_index(drop=True)
else:
    df = df.reset_index(drop=True)

# create targets
df['HI_t+1'] = df.groupby(station_col)[hi_col].shift(-1)
df['HI_t+2'] = df.groupby(station_col)[hi_col].shift(-2)
df_targets = df.dropna(subset=['HI_t+1','HI_t+2'], how='all').copy()

# feature selection
exclude = {station_col, datetime_col, hi_col, 'HI_t+1', 'HI_t+2'}
features = [c for c in df_targets.columns if c not in exclude and pd.api.types.is_numeric_dtype(df_targets[c])]
if not features:
    features = [hi_col]
print(f"Using features: {features}")

# evaluation loop
def compute_metrics_for_horizon(horizon_col):
    stations = df_targets[station_col].unique().tolist()
    results = []
    all_y_true, all_y_pred = [], []
    for st in stations:
        train = df_targets[df_targets[station_col] != st].dropna(subset=features + [horizon_col])
        test = df_targets[df_targets[station_col] == st].dropna(subset=features + [horizon_col])
        if train.empty or test.empty:
            continue
        X_train, y_train = train[features].values, train[horizon_col].values
        X_test, y_test = test[features].values, test[horizon_col].values
        if has_xgb:
            model = XGBRegressor(n_estimators=100, max_depth=6, random_state=42, verbosity=0)
        else:
            model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        r2 = r2_score(y_test, y_pred)
        results.append({'station': st, 'n_test': len(y_test), 'MAE': mae, 'RMSE': rmse, 'R2': r2})
        all_y_true.extend(y_test.tolist())
        all_y_pred.extend(y_pred.tolist())
    if all_y_true:
        overall = {
            'station': 'ALL',
            'n_test': len(all_y_true),
            'MAE': mean_absolute_error(all_y_true, all_y_pred),
            'RMSE': mean_squared_error(all_y_true, all_y_pred, squared=False),
            'R2': r2_score(all_y_true, all_y_pred)
        }
        results.append(overall)
    return pd.DataFrame(results).sort_values(by='station').reset_index(drop=True)

metrics_t1 = compute_metrics_for_horizon('HI_t+1')
metrics_t2 = compute_metrics_for_horizon('HI_t+2')

print("\\n=== Metrics for HI_t+1 (tomorrow) ===")
display(metrics_t1)
print("\\n=== Metrics for HI_t+2 (day after tomorrow) ===")
display(metrics_t2)

general_metrics = pd.DataFrame([
    {'horizon':'t+1', **metrics_t1[metrics_t1['station']=='ALL'].iloc[0].to_dict()},
    {'horizon':'t+2', **metrics_t2[metrics_t2['station']=='ALL'].iloc[0].to_dict()}
]).rename(columns={'n_test':'n_test_overall'})

print("\\n=== General Metrics (all stations combined) ===")
display(general_metrics)

HI_forecast_metrics = {'t+1': metrics_t1, 't+2': metrics_t2, 'general': general_metrics}
